In [ ]:
!pip install moviepy --quiet

In [ ]:
!pip install pytube ffmpeg-python --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.2 MB/s eta 0:00:00


In [ ]:
!pip install asyncffmpeg --quiet

In [ ]:
!pip install asynccpu --quiet

In [ ]:
#!pip install Office365-REST-Python-Client

In [ ]:
#!pip install requests_ntlm

In [ ]:
#!pip uninstall ffmpeg ffmpeg-python

In [ ]:
from pytube import YouTube
import ffmpeg
import requests
from urllib.parse import urlparse
import requests
from bs4 import BeautifulSoup
import json
import os
import warnings
warnings.filterwarnings('ignore')
from moviepy.editor import *
import concurrent.futures
from multiprocessing import process
from threading import Thread
from asynccpu import ProcessTaskPoolExecutor
from asyncffmpeg import FFmpegCoroutineFactory, StreamSpec
import asyncio


In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def is_mp4_playable(file_path):
  try:
      # Probe the file to get information about it
      probe = ffmpeg.probe(file_path)
      # Check if the file format is recognized as video
      if 'streams' in probe and any(stream['codec_type'] == 'video' for stream in probe['streams']):
          print("MP4 file is playable.")
          return True
      else:
          print("MP4 file is corrupt or non-playable.")
          return False
  except ffmpeg.Error as e:
      #print("Error occurred:", e.stderr)
      return False

In [ ]:
def extract_audio(video_url, output_folder):
  if output_folder:
    audio, err = (
      ffmpeg
      .input(video_url)
      .output("pipe:", format='mp3', acodec='libmp3lame', audio_bitrate='320k')
      .run(capture_stdout=True)
    )
    with open(output_folder, 'wb') as f:
        f.write(audio)
    print("Audio extraction complete")
  else:
    print("Audio extraction not done - no output file given")

In [ ]:
def extract_frames(video_url, output_folder, fps=0.2):
  if output_folder:
    (
        ffmpeg
        .input(video_url)
        .filter('fps', fps=fps)
        .output(output_folder + '/frame%d.png', **{'qscale:v': 2})
        .run()
    )
    print("video frames extraction complete")
  else:
    print("Video frames extraction not done - no output file given")

In [ ]:
def process_video(link):

  if len(urlparse(link).netloc) > 0:
    if urlparse(link).netloc == "www.youtube.com":
      yt = YouTube(link)
      video_path = yt.streams[0].url
      return video_path

    elif urlparse(link).netloc == "www.linkedin.com":
      r = requests.get(link)
      soup = BeautifulSoup(r.content, 'html.parser')
      data_linkedin = json.loads(soup.find('script', type='application/ld+json').text)
      if data_linkedin['isAccessibleForFree'] == True:
        video_path = json.loads(soup.video['data-sources'])[0]['src']
        return video_path
      else:
        print("Sorry! Can't process video. Please make sure the video is free for access.")
        return

    elif urlparse(link).netloc == "www.coursera.org":
      r = requests.get(link)
      soup = BeautifulSoup(r.content, 'html.parser')
      data_coursera = json.loads(soup.find('script', type='application/ld+json').text)
      video_path = data_coursera['@graph'][1]['contentURL']
      return video_path

    else:
      print("Sorry! Can't process video. Please make sure the video link is valid.")
      return

  else:
    if(is_mp4_playable(link)):
      video_path = link
      return video_path
    else:
      print("Sorry! Can't process video. Please make sure the video file exists and is not corrupted.")
      return

In [ ]:
async def main(video_link, output_file_path_audio,output_file_path_frames):

  video_path = process_video(video_link)
  if video_path:
      awaitables = [
            asyncio.to_thread(extract_audio, video_path, output_file_path_audio),
            asyncio.to_thread(extract_frames, video_path, output_file_path_frames)
      ]
      await asyncio.gather(*awaitables)

In [ ]:
if __name__ == '__main__':
  video_link=input("Video link: ")
  output_file_path_audio=input("MP3 file path to store audio: ")
  output_file_path_frames=input("Folder path to store video frames: ")
  await main(video_link, output_file_path_audio,output_file_path_frames)

https://www.youtube.com/watch?v=Pu5CNKweuHw
/content/drive/My Drive/LLM_Project/youtube_audio_2.mp3
/content/drive/My Drive/LLM_Project/frames
video frames extraction complete
Audio extraction complete


In [ ]:
# # Invalid URL
# process_video("https://exampleurl",'/content/drive/My Drive/LLM_Project/link_audio.mp3')
# # Youtube
# process_video("https://www.youtube.com/shorts/ThR0tcH37bs",output_file_path_audio='/content/drive/My Drive/LLM_Project/youtube_audio.mp3',
#               output_file_path_frames="/content/drive/My Drive/LLM_Project/video_frames")
# # Linkedin free course
# process_video("https://www.linkedin.com/learning/administrative-professional-weekly-tips/managing-morale-in-your-office?&u=74653650",
#               output_file_path_audio='/content/drive/My Drive/LLM_Project/linkedin_audio_free.mp3',frames_extraction=False)
# # Linkedin non-free course
# process_video("https://www.linkedin.com/learning/tableau-for-data-scientists/solution-connecting-to-data?resume=false&u=74653650",
#               output_file_path_audio='/content/drive/My Drive/LLM_Project/linkedin_audio_paid.mp3',frames_extraction=False)
# # Coursera free course
# process_video("https://www.coursera.org/learn/the-science-of-well-being/lecture/yaNZk/part-2-goal-setting",
#               output_file_path_frames="/content/drive/My Drive/LLM_Project/video_frames",audio_extraction=False)
# # Uploaded MP4 video file - correct
# process_video("/content/drive/My Drive/LLM_Project/LLM_lecture.mp4",output_file_path_audio='/content/drive/My Drive/LLM_Project/uploaded_MP4_audio_1.mp3',frames_extraction=False)
# # Uploaded MP4 video file - path doesn't exist
# process_video("/content/drive/My Drive/LLM_Project/LLM_lecture.mp3",'/content/drive/My Drive/LLM_Project/uploaded_MP4_audio.mp3')
# # Uploaded MP4 video file - corrupt/non-playable file
# process_video("/content/drive/My Drive/LLM_Project/empty_video.mp4",'/content/drive/My Drive/LLM_Project/uploaded_MP4_audio.mp3')